In [6]:
from accounts import jp
# jp.ID, jp.PW

In [31]:
import os
from datetime import datetime
import time
import requests
import re

import pandas as pd

from bs4 import BeautifulSoup

import selenium


from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [67]:
# 크롬으로 잡플래닛 사이트 열기
service = Service(executable_path="chromedriver.exe")
browser = webdriver.Chrome(service=service)

url = r"https://www.jobplanet.co.kr/job"

browser.maximize_window()
print(browser.get_window_size())

browser.get(r"https://www.jobplanet.co.kr/job")

{'width': 1512, 'height': 944}


In [68]:
###########################
# 잡플래닛에 로그인 하기
###########################

# 로그인 버튼 찾기 및 클릭
login_btn = browser.find_element(By.CSS_SELECTOR,"a.btn_txt.login")
login_btn.click()

# 로그인 ID와 PW 쳐야할 곳 클릭 + 적기
login_Id = browser.find_element(By.CSS_SELECTOR, "#user_email")
login_Id.click()
login_Id.send_keys(jp.ID)

login_Pw = browser.find_element(By.CSS_SELECTOR, "#user_password")
login_Pw.click()
login_Pw.send_keys(jp.PW)

# 이메일로 로그인 버튼 누르기
em_log = browser.find_element(By.CSS_SELECTOR, "fieldset > button")
em_log.click()

In [70]:
###########################
# 잡플래닛 검색창에 회사 입력하고 맨 처음에 나오는 기업 클릭하기.
###########################

# 검색창 누르기
search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
search_bar.click()

try:
  # 내가 원하는 회사 입력하기
  want_cp = input()
  search_bar.send_keys(want_cp)
  search_bar.send_keys(Keys.RETURN)
    
  # 검색결과 기다리기.
  time.sleep(2)

  # 맨처음에 나오는 기업 클릭하기
  company = browser.find_element(By.CSS_SELECTOR, "div.is_company_card > div:nth-child(1) > a")
  company.click()

# 회사를 잘못 기입했거나 회사가 없을 경우
except NoSuchElementException:
  # 에러메세지 띄우기
  print("No search result found.")
  break

# 다시 검색창 누르기
search_bar = browser.find_element(By.CSS_SELECTOR, "#search_bar_search_query")
search_bar.click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"div.is_company_card > div:nth-child(1) > a"}
  (Session info: chrome=114.0.5735.198); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102507f48 chromedriver + 4226888
1   chromedriver                        0x00000001025004f4 chromedriver + 4195572
2   chromedriver                        0x0000000102144d68 chromedriver + 281960
3   chromedriver                        0x000000010217ffc8 chromedriver + 524232
4   chromedriver                        0x00000001021b6c58 chromedriver + 748632
5   chromedriver                        0x0000000102173f1c chromedriver + 474908
6   chromedriver                        0x0000000102174ef4 chromedriver + 478964
7   chromedriver                        0x00000001024c959c chromedriver + 3

In [60]:
###########################
# 회사페이지에서 팝업창 모두 닫기.
###########################

# 팝업창이 뜨면 그 팝업창을 없애는 버튼 만들기
try:
    pop_1 = browser.find_element_by_css_selector("div.premium_modal_header > button")
    pop_1.click()
except:
    pop_2 = browser.find_element(By.CSS_SELECTOR, "#jpfollow_induce_panel > button")
    pop_2.click()

In [63]:
section:nth-child(7) > div > div.ctbody_col2 > div > div.us_label_wrap > h2

section:nth-child(4) > div > div.ctbody_col2 > div > div.us_label_wrap > h2

In [66]:
###########################
# 회사페이지에서 리뷰 가져오기.
###########################
def get_jp_company_list():
    """
    잡플래닛 회사 리뷰 상세 내용을 url을 받아서 내용을 반환.
    연결
    """
    # 리뷰들을 리스트로 저장하는 변수
    review_list = []
    # 감지된 숫자 저장하는 변수
    found_numbers = []

    # 무한 루프를 돌기.
    while True:
        
        # 소스 끌어오기.
        html = browser.page_source
        soup = BeautifulSoup(html, "lxml")
        
        # html 내용 가져오기
        review = soup.select("div.us_label_wrap > h2")
        # 페이지 수 가져오기.
        strong_elements = soup.select("article > strong")

        # 텍스트로 추출한 내용을 리스트에 추가해서 넣어주기 위해 이용.
        for text in review : # for문은 for 채워져있는 값의 원소 in 채워져있는 리스트
            review_list.append(text.get_text().strip())
        time.sleep(2)

        # 다음 페이지로 이동하기 위한 버튼을 찾기.
        try:
            next_btn = browser.find_element(By.CSS_SELECTOR,f"a.btn_pgnext")

            # 버튼이 있으면 클릭합니다.
            next_btn.click()

            # 버튼 누르고나서 정보를 가져올때까지 슬립(몇초동안)걸어두기.
            time.sleep(5)
            
        except:
            break

        # found_numbers 변수가 2번 등장하면 while 루프 빠져나오기.
        if not next_btn:
            break

        # 각 요소에 대해:
        for strong_element in strong_elements:
            # 요소 안의 숫자 추출
            number_in_element = int(strong_element.text.strip())

            # 숫자가 found_numbers 변수에 2번 이상 등장했으면 루프를 종료 (break 사용)
            if found_numbers.count(number_in_element) >= 1:
                print(f"Number {number_in_element} appeared twice")
                break
            else:
                # 숫자를 found_numbers 변수에 추가
                found_numbers.append(number_in_element)

    # 브라우저 끄기
    # browser.close()

    # 데이터프레임으로 관리하겠다 지정.
    # return pd.DataFrame({"jp_review":review_list})
# get_jp_company_list()